In [1]:
import cplex
from asignacion_cuadrillas import armar_lp, resolver_lp, mostrar_solucion, cargar_instancia_con_configuracion
from cplex._internal._constants import CPX_VARSEL_DEFAULT

In [4]:
def configurar_y_resolver(path, **config):
    instancia = cargar_instancia_con_configuracion(path, **config)
    prob = cplex.Cplex()
    armar_lp(prob, instancia, shouldWrite=False)
    resolver_lp(prob, instancia)
    mostrar_solucion(prob, instancia)

activar_restriccion_opcional_1 = False, 
activar_restriccion_opcional_2 = False,
tolerancia = 1e-6,
seleccion_nodo = None,
seleccion_variable = None,
heuristica_primal = None,
preproceso = None

In [5]:
configurar_y_resolver("./entradas/entrada2.txt", activar_restriccion_opcional_1 = True)

949
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_MIPGap                   9.9999999999999995e-07
Found incumbent of value 0.000000 after 0.00 sec. (0.10 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 54 rows and 0 columns.
Reduced MIP has 12786 rows, 949 columns, and 63424 nonzeros.
Reduced MIP has 909 binaries, 40 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (49.40 ticks)
Probing time = 0.03 sec. (8.79 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 12786 rows, 949 columns, and 63424 nonzeros.
Reduced MIP has 909 binaries, 40 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (65.36 ticks)
Probing time = 0.02 sec. (8.72 ticks)
Clique table members: 10625.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.06 sec. (55.02 ti